# Merges and matching
* You might have seen -- in the previous example -- that there is more we might want to do. The two files concern the same entities! 

This workbook was based upon [the merges getting started](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#merge) in the Pandas documentation. 

# Data lakes
* a *data lake* is a large set of files with some columns that match. 
* the *merging problem* is to combine data for two or more files to get more complete information about an entity. 
* Here is a compelling example. 
* *The key is to create columns you can match.* 

*In the following, I have shortened printing of large tables. You can remove the shortening at need. *

Let's load our first file from http://data.gov: population of towns in Connecticut. 

In [1]:
import pandas as pd
population = pd.read_csv('2010_Population_By_Town.csv')
population.head()

,Town Num,TOWN,2010 Population
0,1,Andover,3303
1,2,Ansonia,19249
2,3,Ashford,4317
3,4,Avon,18098
4,5,Barkhamsted,3799


This is a table of populations for each town in Connecticut. We also have the following table: 

In [2]:
tax = pd.read_csv('2012_Retail_Sales_By_Town_ALL_NAICS.csv', engine='python', skipfooter=8)
tax.head()

,Municipality,Number of Taxpayers,Total Retail Sales of Goods,Total Tax Due **(Excluding Tax at 9.35% Rate),Tax Due At 6.35%,Tax Due at 7%
0,ANDOVER (001),127,5529457.0,428026.0,427872.0,59.0
1,ANSONIA (002),402,89228175.0,3817640.0,3813289.0,2042.0
2,ASHFORD (003),166,12598684.0,659952.0,654175.0,5653.0
3,AVON (004),739,205383841.0,8690891.0,8575388.0,113441.0
4,BARKHAMSTED (005),174,30403443.0,695373.0,694846.0,0.0


This is a table of sales tax for the same towns. 

# The key observation

* These tables are both about the same towns, but 
* They represent the towns differently. 
* To combine them into one table, we need a common description of the town 

# What's different?
* case of town name. 
* extra stuff in town name. 

# The key strategy
* turn them both into something that matches. 
* lowercase the unadorned name. 
* lowercase and trim the adorned name. 

# Step 1: add lowercase names to `population`

In [3]:
lowercase = population.TOWN.str.lower()
lowercase.head()

0        andover
1        ansonia
2        ashford
3           avon
4    barkhamsted
Name: TOWN, dtype: object

In [4]:
population['lowercase'] = lowercase
population.head()

,Town Num,TOWN,2010 Population,lowercase
0,1,Andover,3303,andover
1,2,Ansonia,19249,ansonia
2,3,Ashford,4317,ashford
3,4,Avon,18098,avon
4,5,Barkhamsted,3799,barkhamsted


# Step 2: transform the name in `tax`
* `tax.Municipality`: the adorned name. 
* `.str.split('(')`: split at '(' character. 
* `[0]`: select first part of split. 
* `.str.strip()`: remove spaces from both sides. 
* `.str.lower()`: lowercase the result. 

In [5]:
lowercase = tax.Municipality.str.split('(', expand=True)[0].str.strip().str.lower()
lowercase.head()


0        andover
1        ansonia
2        ashford
3           avon
4    barkhamsted
Name: 0, dtype: object

In [6]:
tax['lowercase'] = lowercase
tax.head()

,Municipality,Number of Taxpayers,Total Retail Sales of Goods,Total Tax Due **(Excluding Tax at 9.35% Rate),Tax Due At 6.35%,Tax Due at 7%,lowercase
0,ANDOVER (001),127,5529457.0,428026.0,427872.0,59.0,andover
1,ANSONIA (002),402,89228175.0,3817640.0,3813289.0,2042.0,ansonia
2,ASHFORD (003),166,12598684.0,659952.0,654175.0,5653.0,ashford
3,AVON (004),739,205383841.0,8690891.0,8575388.0,113441.0,avon
4,BARKHAMSTED (005),174,30403443.0,695373.0,694846.0,0.0,barkhamsted


# Step 3: merge on now-common column 'lowercase': 
* `left`, `right`: positions of source `DataFrame`s in merged `DataFrame`. 
* `how='outer'`: leave records that don't match in the data. 
* `on=`: what to match

In [7]:
both = pd.merge(left=population, right=tax, how='outer', on='lowercase')
both.head()

,Town Num,TOWN,2010 Population,lowercase,Municipality,Number of Taxpayers,Total Retail Sales of Goods,Total Tax Due **(Excluding Tax at 9.35% Rate),Tax Due At 6.35%,Tax Due at 7%
0,1,Andover,3303,andover,ANDOVER (001),127,5529457.0,428026.0,427872.0,59.0
1,2,Ansonia,19249,ansonia,ANSONIA (002),402,89228175.0,3817640.0,3813289.0,2042.0
2,3,Ashford,4317,ashford,ASHFORD (003),166,12598684.0,659952.0,654175.0,5653.0
3,4,Avon,18098,avon,AVON (004),739,205383841.0,8690891.0,8575388.0,113441.0
4,5,Barkhamsted,3799,barkhamsted,BARKHAMSTED (005),174,30403443.0,695373.0,694846.0,0.0


# A few notes
* We could have used most any mechanism to make columns the same. 
* E.g., uppercase rather than lowercase. 
* The important thing is that they are exactly the same in format. 
* If the same columns are indexes, so much the better. This improves performance. 

# The data merging problem
* https://data.gov is a huge "data lake" of CSV files. 
* Many of them describe the same entities. 
* But they may depict the entities differently, and substantive creativity may be necessary to collect all the data for each entity, in this case, towns in Connecticut. 

# Data fusion
* More generally, there is a problem of *Data Fusion* that goes beyond mere tables.
* Entities can be geospatial, i.e., on a map. 
* Entities can overlap. 
* Data may only be measured for part of an entity, e.g., a county of a state. 
* Data may not be commensurate for the same entity. 

# Often, data fusion is more difficult than the analysis that follows. 
* Column names are synonyms, or missing. 
* Some columns that are named the same contain different data.
* The US 'Open Data Initiative' says that data has to be available, but *does not specify its format or metadata format.* 
* The EU version is even less specific: *it doesn't even specify that data should be machine readable!*
* The [Research Data Alliance](https://rd-alliance.org/) is trying to do something about this by defining metadata and structural standards for CSV data. 
* My own project [HydroShare](https://www.hydroshare.org) goes much farther, and is compliant with rather rigorous metadata and discoverability standards defined by the [DataOne initiative](https://dataone.org). 

# What is `how`? 
* `how`: the join type.
* `'outer'`: leave rows in that don't have matches. 
* `'inner'`: omit any rows that don't match. 
* `'left'`: keep rows on left if they don't match. 
* `'right'`: keep rows on right if they don't match. 
* These names are consistent with names of "join types" in database theory. 
* In fact, that's what we're doing. 

# Let's put this into practice. 

Let's register you for grading purposes. 

In [8]:
# Don't change this cell; just run it. 
from client.api.notebook import Notebook
ok = Notebook('03-05-data-manipulation.ok')

Assignment: 03-05 Data manipulation
OK, version v1.14.15



Then let's create some `DataFrame`s to play with. 

In [9]:
phones = pd.DataFrame({
    'name': ['Mark', 'Anne', 'Frank', 'Lisa'],
    'phone': ['7815551212', 'unlisted', '4035551212', '9195551212']  # NB
})
print("phones:")
print(phones)

addresses = pd.DataFrame({
    'name': ['Frank', 'Anne', 'Mark', 'Samantha'],
    'city': ['Boston', 'Austin', 'Boston', 'Los Angeles'],
    'state': ['MA', 'TX', 'MA', 'CA']
})
print('\naddresses:')
print(addresses)

pets = pd.DataFrame({
    'name': ['Garfield', 'Snoopy', 'Brrf', 'Bill'],
    'type': ['cat', 'dog', 'dog', 'cat'],
    'owner': ['Frank', 'Lisa', 'Samantha', 'Lisa']
})
print('\npets:')
print(pets)

hobbies = pd.DataFrame({
    'name':['Frank', 'Frank', 'George', 'Anne', 'Mark', 'Mark', 'Samantha'], 
    'hobby':['cycling', 'astronomy', 'knitting', 'tennis', 'cycling', 'rock climbing', 'astronomy'],
})
print('\nhobbies:')
print(hobbies)

phones:
    name       phone
0   Mark  7815551212
1   Anne    unlisted
2  Frank  4035551212
3   Lisa  9195551212

addresses:
       name         city state
0     Frank       Boston    MA
1      Anne       Austin    TX
2      Mark       Boston    MA
3  Samantha  Los Angeles    CA

pets:
       name type     owner
0  Garfield  cat     Frank
1    Snoopy  dog      Lisa
2      Brrf  dog  Samantha
3      Bill  cat      Lisa

hobbies:
       name          hobby
0     Frank        cycling
1     Frank      astronomy
2    George       knitting
3      Anne         tennis
4      Mark        cycling
5      Mark  rock climbing
6  Samantha      astronomy


(NB: all the phone numbers are directory assistance. I have learned, in the past, that in any public set of notes, some idiot will actually call the numbers I specify!)

1. Make up a regular address book `book` by combining `addresses` and `phones`. We want an entry even if we don't have a phone or an address for a person. 

In [10]:
# Your answer: 
book = pd.merge(left=addresses, right=phones, how='outer', on='name')
book

,name,city,state,phone
0,Frank,Boston,MA,4035551212
1,Anne,Austin,TX,unlisted
2,Mark,Boston,MA,7815551212
3,Samantha,Los Angeles,CA,NaN
4,Lisa,NaN,NaN,9195551212


In [16]:
_ = ok.grade('q01')  # run to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



2. Merge `pets` with `book` to get an address book `owners` for the `pets`. In this case we leave out an entry if there isn't a pet there. Hint: since column names aren't the same, use `left_on=` and `right_on=`. See [full merge documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) for details.

In [15]:
# Your answer:
owners = pd.merge(left=pets, right=book, how='left', left_on='owner', right_on='name')
owners

,name_x,type,owner,name_y,city,state,phone
0,Garfield,cat,Frank,Frank,Boston,MA,4035551212
1,Snoopy,dog,Lisa,Lisa,NaN,NaN,9195551212
2,Brrf,dog,Samantha,Samantha,Los Angeles,CA,NaN
3,Bill,cat,Lisa,Lisa,NaN,NaN,9195551212


In [17]:
_ = ok.grade('q02')  # run to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



3. Figure out which people share the same hobby by creating a merge `common` of `hobbies` with itself. Then eliminate lines with the same name for both people via row selection.  Also eliminate duplicates by ensuring that names are in alphabetical order from left to right. 

In [20]:
# Your answer: 
common = pd.merge(left=hobbies, right=hobbies, how='inner', on='hobby')
common = common[(common.name_x != common.name_y) & (common.name_x < common.name_y) ]
common

,name_x,hobby,name_y
1,Frank,cycling,Mark
5,Frank,astronomy,Samantha


In [21]:
_ = ok.grade('q03')  # run to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



4. For people in `common`, join with `addresses` and select those that live in the same state. Put the result into `pairs`.

In [22]:
# Your answer: 
p1 = pd.merge(left=common, right=addresses, how='left', left_on='name_x', right_on='name')
p2 = pd.merge(left=p1, right=addresses, how='left', left_on='name_y', right_on='name')
pairs = p2[p2.state_x == p2.state_y]
print(pairs)
pairs

  name_x    hobby name_y name_x  city_x state_x name_y  city_y state_y
0  Frank  cycling   Mark  Frank  Boston      MA   Mark  Boston      MA


,name_x,hobby,name_y,name_x,city_x,state_x,name_y,city_y,state_y
0,Frank,cycling,Mark,Frank,Boston,MA,Mark,Boston,MA


In [23]:
_ = ok.grade('q04')  # run to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



5. Delete extra columns from `pairs` to leave `name_x`, `hobby`, and `name_y`. 
Put the result in `possible`. Hint: google this.

In [24]:
# Your answer: 
possible = pairs.loc[:,~pairs.columns.duplicated()].loc[:,'name_x':'name_y']
print(possible)
possible

  name_x    hobby name_y
0  Frank  cycling   Mark


,name_x,hobby,name_y
0,Frank,cycling,Mark


In [25]:
_ = ok.grade('q05')  # run to check your answer

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed

